## The Battle of the Neighborhoods

The objective of this project is to identify the best corners in the city of Santiago, Chile to place on-field sellers for a flower shop named Crocus. According to the company, the average daily sales of a flower shop show seasonality within a week, increasing by 200% on the weekends. Additionally, profits increase to 9 times the average daily profit for special dates, like Valentine’s Day or Mother’s Day. The company is interested in capturing more of the market in those dates, and to achieve that they plan on placing on-field sellers in 5 corners of certain neighborhoods of Santiago for weekends and special dates. 

To decide which corners are better than others, Crocus has provided a list of 30 potential corners of the city that have shown high foot traffic. The company also requested that the choice is made based on nearby complementary products, such as chocolate shops, restaurants and movie theaters, and competitor location. The final criteria for the selection of corners will be explained in detail in the methodology section.

This project will be carried out in 5 steps: 
1. Import corners data set
2. Search in the Foursquare API for venues and get a dataframe with the results
3. Count the venues by category for each corner
4. Sort the corners according to the criteria
5. Select corners

In this notebook every step will be explained and the code to carry the out will be executed


<b>1. Import corners data set</b>
    
The data set containing the corners to be evaluated was given by the flower shop and is contained in a csv file. The file has a table with 31 rows and 5 columns, the columns store the following information: Corner id number, address, neighborhood, latitude, longitude. Each row represents a corner. In order to import this information it is important to first import the pandas library.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.19 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.17 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  57.39 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.51 MB/s
vincent-0.4.4- 100% |###################

In [2]:
stgo_corners=pd.read_csv('https://raw.githubusercontent.com/mpsapag/Coursera_Capstone/master/Corners%20Santiago.csv')
stgo_corners

,Corner,Address,Neighborhood,Latitude,Longitude
0,1,Plazuela Los Leones,Providencia,-33.419869,-70.605912
1,2,Avenida Nueva Providencia 2200,Providencia,-33.422751,-70.609560
2,3,Avenida Pedro de Valdivia 101,Providencia,-33.424649,-70.611985
3,4,Avenida Suecia 780,Providencia,-33.427532,-70.605505
4,5,Avenida El Bosque 963,Providencia,-33.428427,-70.596128
5,6,Avenida Nueva Providencia 1398,Providencia,-33.428874,-70.618359
6,7,Metro Francisco Bilbao,Providencia,-33.430808,-70.586816
7,8,Latadia 4141,Las Condes,-33.431166,-70.578576
8,9,Avenida Ossa 1552,Ñuñoa,-33.439474,-70.572611
9,10,El Alcalde 15,Las Condes,-33.416337,-70.595227


<b>2.Search in the Foursquare API and get a dataframe with the results</b>

First, in order to get information from Foursquare, the credentials for the account that will be used need to be defined. For the purpose of this project a radious of 300 m will be used, since that is what is considered a person would comfortably walk to buy flowers. The limit will be set to 100 results.

In [3]:
CLIENT_ID = 'F2ELEML3GET2RAUDGCNQMG3Z3YVOR2DDW5YSADCUHOV0JA5W'
CLIENT_SECRET = 'LUNTF0C3EXJN45I33DCG1WWHA21GID5MGG3RPKFPH03NMXZI' 
VERSION = '20180605'
radius=300
LIMIT=100

A function will be defined to search the 300m radious arround each corner, looking for relevant venues. One of the arguments of this function is the query for the search, that will vary, first it will be done to get flower shops, then chocolate shops or candy shops, restaurants and movie theaters. The function will also get a category ID in order to narrow the search to the categories relevant for the analysis. 

In [4]:
def getNearbyVenues(numbers, latitudes, longitudes, search_query, cat_id, radius=300):
    
    venues_list=[]
    for number, lat, lng in zip(numbers, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            search_query,
            radius, 
            LIMIT,
            cat_id)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            number, 
            v['name'], 
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Corner',
                  'Venue',
                  'Venue Category']
    
    return(nearby_venues)

This function will now be used to get the venues that match a search for: Flower Shop, chocolate shop or candy shop, Restaurant and Movie Theater. 

In [5]:
corners_flowers=getNearbyVenues(numbers=stgo_corners['Corner'],
                                latitudes=stgo_corners['Latitude'],
                                longitudes=stgo_corners['Longitude'],
                                search_query='Flower Shop', cat_id='4bf58dd8d48988d11b951735')
corners_candy=getNearbyVenues(numbers=stgo_corners['Corner'],
                                latitudes=stgo_corners['Latitude'],
                                longitudes=stgo_corners['Longitude'],
                                search_query='Chocolate Shop', cat_id='4bf58dd8d48988d117951735')
corners_chocolate=getNearbyVenues(numbers=stgo_corners['Corner'],
                                latitudes=stgo_corners['Latitude'],
                                longitudes=stgo_corners['Longitude'],
                                search_query='Chocolate Shop', cat_id='52f2ab2ebcbc57f1066b8b31')
corners_food=getNearbyVenues(numbers=stgo_corners['Corner'],
                                latitudes=stgo_corners['Latitude'],
                                longitudes=stgo_corners['Longitude'],
                                search_query='Restaurant', cat_id='4d4b7105d754a06374d81259')
corners_movies=getNearbyVenues(numbers=stgo_corners['Corner'],
                                latitudes=stgo_corners['Latitude'],
                                longitudes=stgo_corners['Longitude'],
                                search_query='Cine', cat_id='4bf58dd8d48988d17f941735')

The dataframes obtained will be as follows

In [6]:
corners_movies

,Corner,Venue,Venue Category
0,1,Cineplanet,Multiplex
1,1,Sala Prime Cineplanet,Multiplex
2,1,Cine Planet Sala 3D,Movie Theater
3,1,Cine Arte Tobalaba,Movie Theater
4,1,pasillo 6-8 Cine Hoyts Maipú,Multiplex
5,1,Cineplanet Costanera Center,Movie Theater
6,2,pasillo 6-8 Cine Hoyts Maipú,Multiplex
7,7,Sale De Cine Chack,Indie Movie Theater
8,13,cine Edificio Urbano Plus,Indie Movie Theater
9,14,Cine tiempo,Movie Theater


<b>3. Count the venues by category for each corner</b>

The next step is to group each dataframe to count the amount of venues for each corners, and clean the resulting data frame to obtain a single column with the corresponding name, counting the number of venues for each category. 

In [7]:
flowers_grouped=corners_flowers.groupby('Corner').count()
flowers_grouped.drop('Venue', axis=1, inplace=True)
flowers_grouped.columns=['Flower_Shops']
candy_grouped=corners_candy.groupby('Corner').count()
candy_grouped.drop('Venue', axis=1, inplace=True)
candy_grouped.columns=['Candy_Shops']
chocolate_grouped=corners_chocolate.groupby('Corner').count()
chocolate_grouped.drop('Venue', axis=1, inplace=True)
chocolate_grouped.columns=['Chocolate_Shops']
food_grouped=corners_food.groupby('Corner').count()
food_grouped.drop('Venue', axis=1, inplace=True)
food_grouped.columns=['Restaurants']
movies_grouped=corners_movies.groupby('Corner').count()
movies_grouped.drop('Venue', axis=1, inplace=True)
movies_grouped.columns=['Movie_Theaters']
movies_grouped

,Movie_Theaters
Corner,
1,6
2,1
7,1
13,1
14,1
15,8
17,2
19,4
25,6


Then the 5 dataframes need to be merged into one

In [8]:
merge_flowers = stgo_corners.join(flowers_grouped, on='Corner')
merge_candy = merge_flowers.join(candy_grouped, on='Corner')
merge_chocolate = merge_candy.join(chocolate_grouped, on='Corner')
merge_food=merge_chocolate.join(food_grouped, on='Corner')
merge_final=merge_food.join(movies_grouped, on='Corner')
merge_final

,Corner,Address,Neighborhood,Latitude,Longitude,Flower_Shops,Candy_Shops,Chocolate_Shops,Restaurants,Movie_Theaters
0,1,Plazuela Los Leones,Providencia,-33.419869,-70.605912,NaN,1.0,NaN,31.0,6.0
1,2,Avenida Nueva Providencia 2200,Providencia,-33.422751,-70.609560,NaN,NaN,NaN,43.0,1.0
2,3,Avenida Pedro de Valdivia 101,Providencia,-33.424649,-70.611985,NaN,1.0,NaN,31.0,NaN
3,4,Avenida Suecia 780,Providencia,-33.427532,-70.605505,NaN,NaN,NaN,1.0,NaN
4,5,Avenida El Bosque 963,Providencia,-33.428427,-70.596128,NaN,NaN,NaN,NaN,NaN
5,6,Avenida Nueva Providencia 1398,Providencia,-33.428874,-70.618359,NaN,1.0,NaN,26.0,NaN
6,7,Metro Francisco Bilbao,Providencia,-33.430808,-70.586816,NaN,NaN,NaN,1.0,1.0
7,8,Latadia 4141,Las Condes,-33.431166,-70.578576,NaN,NaN,NaN,2.0,NaN
8,9,Avenida Ossa 1552,Ñuñoa,-33.439474,-70.572611,NaN,NaN,NaN,3.0,NaN
9,10,El Alcalde 15,Las Condes,-33.416337,-70.595227,NaN,NaN,NaN,9.0,NaN


Finally, the candy shops and chocolates shops columns need to be added and the NaN cells need to be turned into zeros

In [9]:
merge_final=merge_final.fillna(0)
merge_final['Chocolate_Stores']=merge_final['Chocolate_Shops']+merge_final['Candy_Shops']
merge_final.drop(['Candy_Shops', 'Chocolate_Shops'], axis=1,inplace=True)

In [10]:
corners_final=merge_final.fillna(0)

In [11]:
corners_final

,Corner,Address,Neighborhood,Latitude,Longitude,Flower_Shops,Restaurants,Movie_Theaters,Chocolate_Stores
0,1,Plazuela Los Leones,Providencia,-33.419869,-70.605912,0.0,31.0,6.0,1.0
1,2,Avenida Nueva Providencia 2200,Providencia,-33.422751,-70.609560,0.0,43.0,1.0,0.0
2,3,Avenida Pedro de Valdivia 101,Providencia,-33.424649,-70.611985,0.0,31.0,0.0,1.0
3,4,Avenida Suecia 780,Providencia,-33.427532,-70.605505,0.0,1.0,0.0,0.0
4,5,Avenida El Bosque 963,Providencia,-33.428427,-70.596128,0.0,0.0,0.0,0.0
5,6,Avenida Nueva Providencia 1398,Providencia,-33.428874,-70.618359,0.0,26.0,0.0,1.0
6,7,Metro Francisco Bilbao,Providencia,-33.430808,-70.586816,0.0,1.0,1.0,0.0
7,8,Latadia 4141,Las Condes,-33.431166,-70.578576,0.0,2.0,0.0,0.0
8,9,Avenida Ossa 1552,Ñuñoa,-33.439474,-70.572611,0.0,3.0,0.0,0.0
9,10,El Alcalde 15,Las Condes,-33.416337,-70.595227,0.0,9.0,0.0,0.0


<b>4. Sort the corners according to the criteria</b>

First the number of flower shops needs to be sorted in ascending order

In [12]:
corners_final.sort_values(by=['Flower_Shops'], inplace=True)

Then the chocolate stores, the restaurants and the movie thaters need to be sorted in that order following a descending order

In [13]:
corners_final.sort_values(by=['Chocolate_Stores', 'Restaurants', 'Movie_Theaters'],ascending=False, inplace=True)

In [14]:
corners_final

,Corner,Address,Neighborhood,Latitude,Longitude,Flower_Shops,Restaurants,Movie_Theaters,Chocolate_Stores
0,1,Plazuela Los Leones,Providencia,-33.419869,-70.605912,0.0,31.0,6.0,1.0
2,3,Avenida Pedro de Valdivia 101,Providencia,-33.424649,-70.611985,0.0,31.0,0.0,1.0
5,6,Avenida Nueva Providencia 1398,Providencia,-33.428874,-70.618359,0.0,26.0,0.0,1.0
24,25,Avenida Suecia 181,Providencia,-33.418879,-70.609346,0.0,22.0,6.0,1.0
21,22,Santa Isabel 400,Providencia,-33.446353,-70.626189,1.0,8.0,0.0,1.0
28,29,Mall Alto Las Condes,Las Condes,-33.390066,-70.547686,1.0,4.0,3.0,1.0
18,19,Avenida Larraín 5862,La Reina,-33.453282,-70.570271,0.0,2.0,4.0,1.0
1,2,Avenida Nueva Providencia 2200,Providencia,-33.422751,-70.609560,0.0,43.0,1.0,0.0
22,23,Avenida Salvador 42,Providencia,-33.433757,-70.626389,0.0,10.0,0.0,0.0
9,10,El Alcalde 15,Las Condes,-33.416337,-70.595227,0.0,9.0,0.0,0.0


<b> 5. Select corners </b>

This just requires the selection of the top 5 corners on the dataframe

In [15]:
corners_final.head(5)

,Corner,Address,Neighborhood,Latitude,Longitude,Flower_Shops,Restaurants,Movie_Theaters,Chocolate_Stores
0,1,Plazuela Los Leones,Providencia,-33.419869,-70.605912,0.0,31.0,6.0,1.0
2,3,Avenida Pedro de Valdivia 101,Providencia,-33.424649,-70.611985,0.0,31.0,0.0,1.0
5,6,Avenida Nueva Providencia 1398,Providencia,-33.428874,-70.618359,0.0,26.0,0.0,1.0
24,25,Avenida Suecia 181,Providencia,-33.418879,-70.609346,0.0,22.0,6.0,1.0
21,22,Santa Isabel 400,Providencia,-33.446353,-70.626189,1.0,8.0,0.0,1.0


Finally these corners can be represented in a map 

In [16]:
# create map
map_corners = folium.Map(location=[-33.420370, -70.578061], zoom_start=13)

# add markers to the map
for lat, lon, corner in zip(corners_final['Latitude'].head(5), corners_final['Longitude'].head(5), corners_final['Corner'].head(5)):
    label = folium.Popup(str(corner), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_corners)
    
for lat, lon, corner in zip(corners_final['Latitude'].tail(25), corners_final['Longitude'].tail(25), corners_final['Corner'].tail(25)):
    label = folium.Popup(str(corner), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_corners)
       
map_corners